In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm

import csv
import os

import sys
sys.path.append("../desirable_undesirable_activities")

import nbimporter
from desirable_undesirable_scoring import (
    compute_overall_score,
    classify_location,
    rural_union_geom,
    desirable_df,
    grocery_df,
    usda_df,
    tract_gdf,
    undesirable_df
)

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
from tqdm import tqdm
import csv

In [3]:
# Define grid bounds and step size
lat_min, lat_max = 30.4, 35.0
lon_min, lon_max = -85.6, -80.8
step = 0.02

In [4]:
output_file = "desirable_undesirable_scores.csv"

In [ ]:
lat_vals = np.arange(lat_min, lat_max, step)
lon_vals = np.arange(lon_min, lon_max, step)
all_pairs = list(np.array(np.meshgrid(lat_vals, lon_vals)).T.reshape(-1, 2))

In [ ]:
if os.path.exists(output_file):
    processed = pd.read_csv(output_file, usecols=["lat", "lon"])
    processed[["lat", "lon"]] = (
    processed[["lat", "lon"]]
    .astype(float)   
    .round(6)
    )  
    processed_set = set(zip(processed["lat"], processed["lon"]))
else:
    processed_set = set()

In [7]:
lat_lon_pairs = [
    (round(lat, 6), round(lon, 6))
    for lat, lon in all_pairs
    if (round(lat, 6), round(lon, 6)) not in processed_set
]

In [8]:
chunk_size = 20
lat_lon_chunks = [lat_lon_pairs[i:i + chunk_size] for i in range(0, len(lat_lon_pairs), chunk_size)]

In [ ]:
def process_chunk(chunk):
    output = []
    for lat, lon in chunk:
        try:
            is_rural = classify_location(lat, lon, rural_union_geom)
            result = compute_overall_score(
                lat, lon, is_rural,
                desirable_df, grocery_df, usda_df, tract_gdf, undesirable_df
            )
            output.append([
                lat, lon,
                result["Desirable Score"],
                result["Total Deductions"],
                result["Overall Score"],
                result["Final Score"]
            ])
        except Exception as e:
            print(f"Error at ({lat}, {lon}): {e}")
    return output

# Open CSV once for append
with open(output_file, "a", newline="") as f:
    writer = csv.writer(f)
    # Run in parallel with loky + a live tqdm bar
    with tqdm_joblib(tqdm(total=len(lat_lon_chunks), desc="Processing chunks")):
        results = Parallel(
            n_jobs=4,
            backend="loky",
            pre_dispatch="4"   
        )(
            delayed(process_chunk)(chunk)
            for chunk in lat_lon_chunks
        )

    for rows in results:
        writer.writerows(rows)
        f.flush()

/opt/homebrew/anaconda3/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
Processing chunks:   0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point  

In [14]:
ga_tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp").to_crs(epsg=3857)

In [15]:
df = pd.read_csv("../../data/map/desirable_undesirable_activities/desirable_undesirable_scores.csv")          # must contain latitude & longitude columns

gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.lon, df.lat),
        crs="EPSG:4326"                       # 
)

In [16]:
ga_tracts = ga_tracts.to_crs(gdf.crs)


In [17]:
ga_boundary = ga_tracts.dissolve()

# Spatial filter: keep only points inside Georgia
gdf_ga_only = gdf[gdf.within(ga_boundary.geometry.iloc[0])]

In [18]:
gdf_ga_only.to_file("../../data/map/desirable_undesirable_activities/desirable_undesirable_scores.geojson", driver="GeoJSON")